In [ ]:
# NBVAL_SKIP
from rubix.telescope.filters.filters import load_filter, print_filter_list, print_filter_list_info, print_filter_property

In [ ]:
# NBVAL_SKIP
print_filter_list("SLOAN")

In [ ]:
# NBVAL_SKIP
print_filter_list_info("SLOAN")

In [ ]:
# NBVAL_SKIP
print_filter_property("SLOAN", "SDSS.u")

In [ ]:
# NBVAL_SKIP
print_filter_property("JWST", "F070W", "NIRCam")

In [ ]:
# NBVAL_SKIP
# load all fliter curves for SLOAN
curves = load_filter("SLOAN")

In [ ]:
# NBVAL_SKIP
curves.filters

In [ ]:
# NBVAL_SKIP
curves.plot()

In [ ]:
# NBVAL_SKIP
filter = curves[1]
filter.plot()

In [ ]:
# NBVAL_SKIP
# cube = 
# here goes the loading of a dummy galaxy data cube. so we can test convolve with the filter

In [ ]:
#NBVAL_SKIP
import matplotlib.pyplot as plt
from rubix.core.pipeline import RubixPipeline 
import os
config = {
    "pipeline":{"name": "calc_ifu"},
    
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
    "data": {
        "name": "IllustrisAPI",
        "args": {
            "api_key": os.environ.get("ILLUSTRIS_API_KEY"),
            "particle_type": ["stars"],
            "simulation": "TNG50-1",
            "snapshot": 99,
            "save_data_path": "data",
        },
        
        "load_galaxy_args": {
        "id": 14,
        "reuse": True,
        },
        
        "subset": {
            "use_subset": True,
            "subset_size": 1000,
        },
    },
    "simulation": {
        "name": "IllustrisTNG",
        "args": {
            "path": "data/galaxy-id-14.hdf5",
        },
    
    },
    "output_path": "output",

    "telescope":
        {"name": "MUSE",
         "psf": {"name": "gaussian", "size": 5, "sigma": 0.6},
         "lsf": {"sigma": 0.5},
         "noise": {"signal_to_noise": 1,"noise_distribution": "normal"},},
    "cosmology":
        {"name": "PLANCK15"},
        
    "galaxy":
        {"dist_z": 0.1,
         "rotation": {"type": "edge-on"},
        },
        
    "ssp": {
        "template": {
            "name": "BruzualCharlot2003"
        },
    },        
}

pipe = RubixPipeline(config)

data = pipe.run()

datacube = data["datacube"]
wave = pipe.telescope.wave_seq

In [ ]:
# NBVAL_SKIP
datacube.shape

In [ ]:
# NBVAL_SKIP
from rubix.telescope.filters.filters import convolve_filter_with_spectra

In [ ]:
# NBVAL_SKIP
filter = curves[1]

In [ ]:
# NBVAL_SKIP
convolved = convolve_filter_with_spectra(filter, datacube, wave)
print(convolved.shape)

In [ ]:
# NBVAL_SKIP
import matplotlib.pyplot as plt
plt.imshow(convolved)
plt.colorbar()

In [ ]:
# NBVAL_SKIP
for filter in curves:
    convolved = convolve_filter_with_spectra(filter, datacube, wave)
    plt.figure()
    plt.imshow(convolved)
    plt.colorbar()
    plt.title(filter.name)

In [ ]:
# NBVAL_SKIP
filters,images =curves.apply_filter_curves(datacube, wave).values()

In [ ]:
# NBVAL_SKIP
filters

In [ ]:
# NBVAL_SKIP
for i,name in zip(images, filters):
    plt.figure()
    plt.imshow(i)
    plt.colorbar()
    plt.title(name)

In [ ]:
# NBVAL_SKIP
# Create an RGB image
# Normalize the images
import numpy as np

def normalize(image):
    image_min = image.min()
    image_max = image.max()
    return (image - image_min) / (image_max - image_min)

r = images[1]
g = images[2]
b = images[3]

rgb = np.stack([r,g,b], axis=-1)

rgb = normalize(rgb)

plt.imshow(rgb)

